In [18]:
%reload_ext autoreload
%autoreload 2


In [19]:
import src.config as config

In [20]:
from datetime import datetime, timedelta 

import pandas as pd 
current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'current date: {current_date}')
# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_date_to = current_date 
fetch_date_from = fetch_date_to - timedelta(days=70) #28 days of data

current date: 2024-08-08 16:00:00


In [21]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # # download 2 files from website
    # rides = load_raw_data(year=from_date_.year) #, months=from_date_.month)
    # rides = rides[(rides.pickup_datetime >= from_date_) & (rides.pickup_datetime < to_date_)]
   
    # rides_2 = load_raw_data(year=to_date_.year) #, months=to_date_.month)
    # rides_2 = rides_2[(rides_2.pickup_datetime < to_date_) & (rides_2.pickup_datetime < to_date_)]

    # rides = pd.concat([rides, rides_2]) 

    # Intenta cargar los datos del primer año
    rides = load_raw_data(year=from_date_.year)
    rides = rides[(rides.pickup_datetime >= from_date_) & (rides.pickup_datetime < to_date_)]

    # Verifica si los años son diferentes
    if from_date_.year != to_date_.year:
    # Carga los datos del segundo año
        rides_2 = load_raw_data(year=to_date_.year)
        rides_2 = rides_2[(rides_2.pickup_datetime >= from_date_) & (rides_2.pickup_datetime < to_date_)]
        rides = pd.concat([rides, rides_2]) 
       # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [22]:
rides = fetch_batch_raw_data(from_date=fetch_date_from, to_date=fetch_date_to)

from_date=Timestamp('2024-05-30 16:00:00'), to_date_=Timestamp('2023-08-10 16:00:00')
File 2023 was already in local storage


In [23]:
len(rides.pickup_location_id.unique())

339

In [24]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 339/339 [00:04<00:00, 71.66it/s]


In [25]:
ts_data.head()

,pickup_hour,rides,pickup_location_id
0,2024-05-30 16:00:00,12,3
1,2024-05-30 17:00:00,6,3
2,2024-05-30 18:00:00,5,3
3,2024-05-30 19:00:00,1,3
4,2024-05-30 20:00:00,0,3


In [26]:
ts_data.tail(5)

,pickup_hour,rides,pickup_location_id
562795,2024-08-08 11:00:00,0,412
562796,2024-08-08 12:00:00,1,412
562797,2024-08-08 13:00:00,0,412
562798,2024-08-08 14:00:00,3,412
562799,2024-08-08 15:00:00,0,412


In [27]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/881813
Connected. Call `.close()` to terminate connection gracefully.


In [28]:
feature_group.insert(ts_data, write_options={'wait_for_job': True})

Uploading Dataframe: 0.00% |          | Rows 0/537600 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/881813/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x26fb42d1f70>, None)